<a href="https://colab.research.google.com/github/0somens/Analysis/blob/main/Fruits_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clasificación de frutas

In [ ]:
%pip install tensorflow

In [ ]:
%pip install streamlit matplotlib numpy pandas

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import kagglehub

In [ ]:
fruits_df = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")



Using Colab cache for faster access to the 'fruit-and-vegetable-image-recognition' dataset.


In [ ]:
print("Path to dataset files:", fruits_df)

Path to dataset files: /kaggle/input/fruit-and-vegetable-image-recognition


In [ ]:
data_dir = fruits_df
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
validation_dir = os.path.join(data_dir, 'validation')

IMG_HEIGHT = 180
IMG_WIDTH = 180
BATCH_SIZE = 32

print(f"Base Data Directory: {data_dir}")
print(f"Training Directory: {train_dir}")
print(f"Testing Directory: {test_dir}")
print(f"Validation Directory: {validation_dir}")
print(f"Image Height: {IMG_HEIGHT}")
print(f"Image Width: {IMG_WIDTH}")
print(f"Batch Size: {BATCH_SIZE}")

Base Data Directory: /kaggle/input/fruit-and-vegetable-image-recognition
Training Directory: /kaggle/input/fruit-and-vegetable-image-recognition/train
Testing Directory: /kaggle/input/fruit-and-vegetable-image-recognition/test
Validation Directory: /kaggle/input/fruit-and-vegetable-image-recognition/validation
Image Height: 180
Image Width: 180
Batch Size: 32


In [ ]:
data_train = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    validation_split=False,

)

Found 3115 files belonging to 36 classes.


In [ ]:
data_cat = data_train.class_names

In [ ]:
data_val = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    validation_split=False,
)

In [ ]:
data_test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=True,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    validation_split=False,
)

In [ ]:
plt.figure(figsize=(10,10))
for image,labels in data_train.take(1):
  for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(data_cat[labels[i]])
    plt.axis('off')

In [ ]:
data_train

In [ ]:
model = Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)), #colores
    # first layer
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    # second layer
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    # third layer
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(data_cat))

])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
epochs = 25
model.fit(data_train, validation_data=data_val, epochs=epochs, batch_size=BATCH_SIZE, verbose=1)


In [ ]:
epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1,2,1)
plt.plot(epochs_range, model.history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, model.history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, model.history.history['loss'], label='Training loss')
plt.plot(epochs_range, model.history.history['val_loss'], label='Validation loss')
plt.title('Loss')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
imagen = '/content/drive/MyDrive/BaseDeDatos/banana.jpg'
image = tf.keras.utils.load_img(imagen, target_size=(IMG_HEIGHT, IMG_WIDTH))
img_arr = tf.keras.utils.img_to_array(image)
img_bat = tf.expand_dims(img_arr, 0)


In [ ]:
predictions = model.predict(img_bat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


In [ ]:
score = tf.nn.softmax(predictions)
print('Veg/Fruit image is {}  with accuracy of {:0.2f}'.format(data_cat[np.argmax(score)], 100*np.max(score)))

Veg/Fruit image is pear  with accuracy of 95.73


In [ ]:
model.save('Image_classify.keras')